In [ ]:
import logging
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from copy import deepcopy
import warnings
import numpy as np
from numpy import pi
import matplotlib.pyplot as plt
from qiskit.circuit import Gate
from qiskit import transpile
from qiskit.transpiler import InstructionProperties
import sys
sys.path.append("/scr/dataset/yuchen/qec_calib/Calib_Compiler")
from qiskit_utilities import (
    save_job_data,
    setup_logger,
    retrieve_expdata,
    get_cr_schedule,
    iy_drag_calibration,
    get_default_cr_params,
    read_calibration_data,
    create_direct_cnot_schedule,
    rough_rz0_correction_calibration,
    fine_rz0_correction_calibration,
    initilize_qiskit_dynamics_backend,
    parallel_rz0_correction_calibration,
    parallel_fine_rz0_correction_calibration,
    parallel_cr_pulse_calib
)
token = "REPLACE WITH IBM TOKEN"
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService(
    token = token,
    channel="ibm_quantum"
)
backend = service.backend("ibm_rensselaer")
setup_logger(filename=None, level=logging.INFO, stdout=True)
logger = logging.getLogger("qiskit_utilities")

In [ ]:
# qubit_pairs = [[117, 116], [123, 122], [64, 65], [0, 1]]
# qubit_pairs = [[0, 14], [3, 2], [5, 6], [16, 8], [10, 11], [22, 15], [17, 30], [20, 19], [24, 25], [27, 28], [36, 32], [38, 37], [41, 40], [43, 44], [47, 46], [55, 49], [64, 54], [57, 56], [59, 60], [62, 72], [67, 66], [69, 70], [77, 71], [75, 90], [91, 79], [83, 82], [86, 85], [88, 89], [106, 93], [96, 95], [99, 100], [102, 103], [112, 108], [111, 122], [113, 114], [117, 118], [125, 124]]
qubit_pairs = [[0, 14], [3, 2], [5, 6], [16, 8], [10, 11], [22, 15], [17, 30], [20, 19], [24, 25], [27, 28], [36, 32], [38, 37], [41, 40], [43, 44], [47, 46], [55, 49], [64, 54], [57, 56], [59, 60], [67, 66], [69, 70], [77, 71], [75, 90], [91, 79], [83, 82], [86, 85], [88, 89], [106, 93], [96, 95], [99, 100], [102, 103], [112, 108], [111, 122], [113, 114], [117, 118], [125, 124]]
qubit_pairs = qubit_pairs[:7]
# qubit_pairs = [[0, 14]]
# qubit_pairs = [qubit_pairs[19]]
initial_calibration_data_lst = []
for qubit_pair in qubit_pairs:
    ratio = 2
    QUBIT_C, QUBIT_T = qubit_pair[0], qubit_pair[1]
    cr_config, ix_config = get_default_cr_params(backend, qubit_pair[0], qubit_pair[1])
    f0 = backend.properties().frequency(QUBIT_C)
    f1 = backend.properties().frequency(QUBIT_T)
    a0 = backend.properties().qubit_property(QUBIT_C)["anharmonicity"][0]
    a1 = backend.properties().qubit_property(QUBIT_T)["anharmonicity"][0]
    duration = backend.defaults().instruction_schedule_map.get("ecr", (QUBIT_C, QUBIT_T)).duration/16 * 4/ratio
    params = {
        "order": "3",
        "t_r": 10,
        "drag_type": "exact",
        "duration": duration,
        "amp": cr_config["amp"]*ratio,
        "angle": cr_config["angle"],
        "Delta": (backend.properties().frequency(QUBIT_C) - backend.properties().frequency(QUBIT_T)) * 1.0e-9 * 2 * pi,
        "a1": 2 * pi * backend.properties().qubit_property(QUBIT_C)["anharmonicity"][0] * 1.0e-9,
        "drag_scale": [1., 1., 1.]
    }
    cr_params = deepcopy(params)
    ix_params = deepcopy(params)
    ix_params["amp"] = ix_config["amp"]*ratio
    ix_params["angle"] = ix_config["angle"]
    ix_params["order"] = "2"
    ix_params["drag_type"] = "01"
    initial_calibration_data = {
        "cr_params": cr_params,
        "ix_params": ix_params,
        "frequency_offset": 0.,
    }
    initial_calibration_data_lst.append(initial_calibration_data)
cr_times = 16 * np.arange(16, duration + 16, duration // 30, dtype=int)
gate_name = "CR-recursive-tr10-direct"

In [ ]:
from qiskit_utilities import parallel_cr_pulse_calib
finished = parallel_cr_pulse_calib(
    qubit_pairs,
    backend=backend,
    cr_times=cr_times,
    initial_calibration_data=initial_calibration_data_lst,
    gate_name=gate_name,
    verbose=True,
    restart=True,
    max_repeat=4,
    rerun_last_calibration=True,
    shots=2048,
    mode="CR"
)

In [ ]:
duration = backend.defaults().instruction_schedule_map.get("ecr", (QUBIT_C, QUBIT_T)).duration/16 * 4 / ratio / 2
cr_times = 16 * np.arange(16, duration + 16, duration//30, dtype=int)

In [ ]:
finished = parallel_cr_pulse_calib(
    finished,
    backend=backend,
    cr_times=cr_times,
    gate_name=gate_name,
    initial_calibration_data=None,
    verbose=True,
    threshold_MHz=0.015,
    restart=True,
    rerun_last_calibration=False,
    max_repeat=3,
    shots=8096,
    mode="IX-pi"
    )

In [ ]:
from qiskit_utilities import parallel_rz0_correction_calibration
parallel_rz0_correction_calibration(
    backend,
    finished,
    gate_name
)

In [ ]:
from qiskit_utilities import parallel_fine_rz0_correction_calibration
parallel_fine_rz0_correction_calibration(
    backend,
    finished,
    gate_name,
    max_repeat_cnot=6
)